In [2]:
import torch
import sys
sys.path.append('/home/zjy/project/MetaIM')
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=7)

In [3]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index

In [4]:
import numpy as np
individual_infection_path = pwd+'/for_meta/cora_individual_infection_sir_200_sample_500.npy'
seeds_infection_path = pwd+'/for_meta/cora_seed_infection_sir_200_sample_500.npy'

individual_infection = np.load(individual_infection_path)
seeds_infection = np.load(seeds_infection_path)
individual_infection.shape,seeds_infection.shape

((2708, 2708), (1000, 2, 2708))

In [5]:
import torch
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp

# 转换为 scipy 稀疏矩阵
adj = to_scipy_sparse_matrix(edge_index)


# def normalize_adj(mx):
#     """Row-normalize sparse matrix"""
#     rowsum = np.array(mx.sum(1))
#     r_inv_sqrt = np.power(rowsum, -0.5).flatten()
#     r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
#     r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
#     return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)



# adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
# adj = normalize_adj(adj + sp.eye(adj.shape[0]))
adj = torch.Tensor(adj.toarray()).to_sparse()
adj


tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)

In [6]:
feat_num = int(seeds_infection[0][0].sum())

In [7]:
from torch.utils.data import Dataset, DataLoader, random_split


class CustomDataset(Dataset):
    def __init__(self, individual_infection,seeds_infection, feat_num):
        self.individual_infection = individual_infection
        self.seeds_infection = seeds_infection
        self.feat_shape = (len(individual_infection), feat_num)

    def __len__(self):
        return len(self.seeds_infection)

    def __getitem__(self, idx):
        seeds= np.nonzero(self.seeds_infection[idx][0])[0]
        
        feature = torch.zeros(self.feat_shape[0],self.feat_shape[1])
        for i in range(len(seeds)):
            seed_i_infection = torch.tensor(self.individual_infection[seeds[i]])
            feature[:, i] = seed_i_infection
            
        label = self.seeds_infection[idx][1]
        
        return self.seeds_infection[idx][0], feature, label

dataset = CustomDataset(individual_infection, seeds_infection, feat_num)

In [8]:
# 定义划分比例
train_ratio = 0.8
test_ratio = 0.2

# 划分数据集
train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*train_ratio), int(len(dataset)*test_ratio)])

train_batch_size = 32
test_batch_size = 2

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

训练VAE

In [9]:
# from data import model 
from data.model.model import VAEModel, Encoder, Decoder
from torch.optim import Adam, SGD
import torch.nn.functional as F

# # hidden_dim = 256
# # latent_dim = 64
hidden_dim = 1024
latent_dim = 64

encoder = Encoder(input_dim= len(seeds_infection[0][0]), 
                  hidden_dim=hidden_dim, 
                  latent_dim=latent_dim)

decoder = Decoder(input_dim=latent_dim, 
                  latent_dim=latent_dim, 
                  hidden_dim=hidden_dim, 
                  output_dim=len(seeds_infection[0][0]))

vae_model = VAEModel(Encoder=encoder, Decoder=decoder).to(device)

optimizer_vae = Adam([{'params': vae_model.parameters()}], 
                 lr=1e-3)
vae_model.train()

VAEModel(
  (Encoder): Encoder(
    (FC_input): Linear(in_features=2708, out_features=1024, bias=True)
    (FC_input2): Linear(in_features=1024, out_features=1024, bias=True)
    (FC_output): Linear(in_features=1024, out_features=64, bias=True)
    (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Decoder): Decoder(
    (FC_input): Linear(in_features=64, out_features=64, bias=True)
    (FC_hidden_1): Linear(in_features=64, out_features=1024, bias=True)
    (FC_hidden_2): Linear(in_features=1024, out_features=1024, bias=True)
    (FC_output): Linear(in_features=1024, out_features=2708, bias=True)
  )
)

In [10]:
for epoch in range(300):
    train_vae_loss = 0
    mean_train_accuracy = 0
    for batch_idx, x_feature_label in enumerate(train_loader):        
        x = x_feature_label[0].to(device)
        optimizer_vae.zero_grad()
        loss = 0
        for i, x_i in enumerate(x):
            x_hat = vae_model(x_i)

            reproduction_loss = F.binary_cross_entropy(x_hat, x_i, reduction='sum')   
            loss += reproduction_loss    
        train_vae_loss += loss.item()
        loss = loss/x.size(0)
        loss.backward()
        optimizer_vae.step()
        
    print("Epoch: {}".format(epoch+1), 
        "\tTrain_vae_loss: {:.4f}".format(train_vae_loss / train_batch_size),
        )
    
for param in vae_model.parameters():
    param.requires_grad = False 

Epoch: 1 	Train_vae_loss: 21576.0738
Epoch: 2 	Train_vae_loss: 14080.8331
Epoch: 3 	Train_vae_loss: 13622.1166
Epoch: 4 	Train_vae_loss: 13540.8799
Epoch: 5 	Train_vae_loss: 13529.3853
Epoch: 6 	Train_vae_loss: 13522.3352
Epoch: 7 	Train_vae_loss: 13515.4281
Epoch: 8 	Train_vae_loss: 13502.3503
Epoch: 9 	Train_vae_loss: 13496.3469
Epoch: 10 	Train_vae_loss: 13492.5474
Epoch: 11 	Train_vae_loss: 13489.6962
Epoch: 12 	Train_vae_loss: 13487.5085
Epoch: 13 	Train_vae_loss: 13483.7418
Epoch: 14 	Train_vae_loss: 13483.7582
Epoch: 15 	Train_vae_loss: 13477.3594
Epoch: 16 	Train_vae_loss: 13476.9675
Epoch: 17 	Train_vae_loss: 13468.3070
Epoch: 18 	Train_vae_loss: 13469.7218
Epoch: 19 	Train_vae_loss: 13468.3749
Epoch: 20 	Train_vae_loss: 13464.0926
Epoch: 21 	Train_vae_loss: 13465.5825
Epoch: 22 	Train_vae_loss: 13461.8043
Epoch: 23 	Train_vae_loss: 13460.8271
Epoch: 24 	Train_vae_loss: 13456.4974
Epoch: 25 	Train_vae_loss: 13457.8939
Epoch: 26 	Train_vae_loss: 13452.9500
Epoch: 27 	Train_vae_

In [11]:
import torch
import torch.nn as nn

from torch_geometric.nn import GATConv
from torch.optim import Adam, SGD
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.linear1 = nn.Linear(latent_dim, latent_dim)
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads)
        self.conv2 = GATConv(hidden_channels * num_heads, out_channels, heads=1)

    def forward(self, feat_x_i, edge_index):
        # x_i = F.sigmoid(self.linear1(x_i))
        # x =  torch.cat((feat_i, x_i), dim=-1)
        x = self.conv1(feat_x_i, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        return F.relu(x)

In [12]:
# from data.model.gat import GAT, SpGAT

# forward_model = SpGAT(nfeat=feat_num, 
#                 nhid=64, 
#                 nclass=1, 
#                 dropout=0.2, 
#                 nheads=1, 
#                 alpha=0.2)

feat_num += latent_dim

forward_model = GAT(feat_num, 512, 1, 4)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=0.001)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

GAT(
  (linear1): Linear(in_features=64, out_features=64, bias=True)
  (conv1): GATConv(199, 512, heads=4)
  (conv2): GATConv(2048, 1, heads=1)
)

In [13]:
edge_index = edge_index.to(device)
top_num = 500
encoder = vae_model.Encoder

for epoch in range(2000):

    total_overall = 0
    forward_loss = 0

    mean_train_accuracy = 0
    for batch_idx, x_feature_label in enumerate(train_loader):  
        x =  x_feature_label[0].to(device)     
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        optimizer.zero_grad()
        
        loss = 0
        train_accuracy = 0
        for i, feat_i in enumerate(features):
            
            # log_sum = torch.sum(torch.log(1 - x_i), dim=1, keepdim=True)
            # feat = 1 - torch.exp(log_sum)
            x_i = x[i]
            x_i = encoder(x_i).detach()
            x_i = x_i.expand(features.shape[1], -1)
            final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            
            
            y_i = labels[i]
            y_hat = forward_model(final_feat_i, edge_index)
            _, top_indices_true = torch.topk(y_i.clone(), top_num)
            label_2 = torch.zeros(y_i.shape).to(device)
            label_2[top_indices_true] = 1
            
            _, top_indices_predict = torch.topk(y_hat.clone().squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            accuracy_i = len(intersection) / top_num       
            train_accuracy += accuracy_i 

            forward_loss = 0.5*F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum') + F.mse_loss(y_hat.squeeze(-1), label_2, reduction='sum')    
            loss += forward_loss    
        
        total_overall += loss.item()    
        train_accuracy /= len(features)
        mean_train_accuracy = train_accuracy
        loss = loss/features.size(0)
        loss.backward()
        optimizer.step()
        # for p in forward_model.parameters():
        #     p.data.clamp_(min=0)
        
        
    print("Epoch: {}".format(epoch+1), 
        "\tTotal: {:.4f}".format(total_overall / train_batch_size),
        "\tMean_train_accuracy: {:.4f}".format(mean_train_accuracy),
        )  
    
    mean_accuracy = 0
    mean_accuracy_sum = 0

    
    for batch_idx, x_feature_label in enumerate(test_loader):  
        x =  x_feature_label[0].to(device)
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        
        accuracy = 0
        accuracy_sum = 0
        
        for i, feat_i in enumerate(features):
            x_i = x[i]
            x_i = encoder(x_i).detach()
            x_i = x_i.expand(features.shape[1], -1)
            final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            y_i = labels[i]
            _, top_indices_true = torch.topk(y_i, top_num)
            
            y_hat = forward_model(final_feat_i, edge_index)
            
            _, top_indices_predict = torch.topk(y_hat.squeeze(-1), top_num)
            
            sum_pre = torch.sum(feat_i, dim=1, keepdim=True)
            _, top_indices_sum = torch.topk(sum_pre.squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()
            
            list_sum = top_indices_sum.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            
            intersection_sum = list(set(list1) & set(list_sum))
            
            accuracy_i = len(intersection) / top_num       
            accuracy += accuracy_i 
            accuracy_sum += len(intersection_sum) / top_num  
        accuracy /= test_batch_size
        accuracy_sum/= test_batch_size
        mean_accuracy = accuracy
        mean_accuracy_sum = accuracy_sum
        break
    
    print(
        "\tMean_test_accuracy: {:.4f}".format(mean_accuracy),
        "\tMean_test_accuracy_sum: {:.4f}".format(mean_accuracy_sum)
        )  

    

Epoch: 1 	Total: 133767.4506 	Mean_train_accuracy: 0.2022
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 2 	Total: 14498.1782 	Mean_train_accuracy: 0.2055
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 3 	Total: 14498.1783 	Mean_train_accuracy: 0.2091
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 4 	Total: 14498.1783 	Mean_train_accuracy: 0.2026
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 5 	Total: 14498.1779 	Mean_train_accuracy: 0.2034
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 6 	Total: 14498.1780 	Mean_train_accuracy: 0.2077
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 7 	Total: 14498.1785 	Mean_train_accuracy: 0.2058
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 8 	Total: 14498.1779 	Mean_train_accuracy: 0.2041
	Mean_test_accuracy: 0.2150 	Mean_test_accuracy_sum: 0.5100
Epoch: 9 	Total: 14498.1779 	Mean_train_accuracy: 0.1993
	Mean_

KeyboardInterrupt: 